In [1]:
import torch
from torch_geometric.datasets import Planetoid
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

# Loading the Cora dataset
dataset = Planetoid(root='data/Planetoid', name='Cora')

Processing...
Done!


In [2]:
class CustomGNN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(CustomGNN, self).__init__()
        self.layer1 = GCNConv(input_dim, hidden_dim)
        self.layer2 = GCNConv(hidden_dim, output_dim)

    def forward(self, feature_data, edge_info):
        # First GCN layer
        x = self.layer1(feature_data, edge_info)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        # Second GCN layer
        x = self.layer2(x, edge_info)
        return F.log_softmax(x, dim=1)

# Initialize the GNN model
input_features = dataset.num_node_features
num_classes = dataset.num_classes
model = CustomGNN(input_features, 16, num_classes)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

graph_data = dataset[0]  # Get the graph data

In [3]:
def train_model():
    model.train()
    optimizer.zero_grad()
    output = model(graph_data.x, graph_data.edge_index)
    loss = F.nll_loss(output[graph_data.train_mask], graph_data.y[graph_data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

for epoch in range(200):
    loss_value = train_model()
    print(f'Epoch: {epoch+1:03d}, Loss: {loss_value:.4f}')

Epoch: 001, Loss: 1.9511
Epoch: 002, Loss: 1.8479
Epoch: 003, Loss: 1.7581
Epoch: 004, Loss: 1.6281
Epoch: 005, Loss: 1.4938
Epoch: 006, Loss: 1.3339
Epoch: 007, Loss: 1.2112
Epoch: 008, Loss: 1.0623
Epoch: 009, Loss: 0.9959
Epoch: 010, Loss: 0.8929
Epoch: 011, Loss: 0.7593
Epoch: 012, Loss: 0.7239
Epoch: 013, Loss: 0.5820
Epoch: 014, Loss: 0.5342
Epoch: 015, Loss: 0.5028
Epoch: 016, Loss: 0.4282
Epoch: 017, Loss: 0.3574
Epoch: 018, Loss: 0.3155
Epoch: 019, Loss: 0.3365
Epoch: 020, Loss: 0.2325
Epoch: 021, Loss: 0.2334
Epoch: 022, Loss: 0.1948
Epoch: 023, Loss: 0.1919
Epoch: 024, Loss: 0.1619
Epoch: 025, Loss: 0.1406
Epoch: 026, Loss: 0.1358
Epoch: 027, Loss: 0.1479
Epoch: 028, Loss: 0.0981
Epoch: 029, Loss: 0.1298
Epoch: 030, Loss: 0.0979
Epoch: 031, Loss: 0.0930
Epoch: 032, Loss: 0.1007
Epoch: 033, Loss: 0.0852
Epoch: 034, Loss: 0.1159
Epoch: 035, Loss: 0.0800
Epoch: 036, Loss: 0.0809
Epoch: 037, Loss: 0.0896
Epoch: 038, Loss: 0.0854
Epoch: 039, Loss: 0.0783
Epoch: 040, Loss: 0.0641


In [4]:
def evaluate_model():
    model.eval()
    with torch.no_grad():
        predictions = model(graph_data.x, graph_data.edge_index).argmax(dim=1)
        correct = (predictions[graph_data.test_mask] == graph_data.y[graph_data.test_mask]).sum()
        acc = int(correct) / int(graph_data.test_mask.sum())
    return acc

accuracy = evaluate_model()
print(f'Test Accuracy: {accuracy:.4f}')

Test Accuracy: 0.8220
